# Models as layers
1 pora pretrained-model uthake layer ke tarah use kerley.

Ismai bhe layer sharing ke tarah layers ke sath us wakt ke weights bhe atay hai(use hote hai).

Calling an instance, whether it’s a layer
instance or a model instance, will always reuse the existing learned representations of
the instance

In [ ]:
# This means you can call a model on an input tensor and retrieve an output tensor:
y = model(x) # for eg this is a model

#If the model has multiple input tensors and multiple output tensors, it should be called with a list of tensors:
y1, y2 = model([x1, x2])

# When you call a model instance, you’re reusing the weights of the model

One simple practical example of what you can build by reusing a model instance is
a vision model that uses a dual camera as its input: two parallel cameras, a few centimeters (one inch) apart. Such a model can perceive depth, which can be useful in
many applications. You shouldn’t need two independent models to extract visual features from the left camera and the right camera before merging the two feeds.
Such low-level processing can be shared across the two inputs: that is, done via layers
that use the same weights and thus share the same representations. Here’s how you’d
implement a Siamese vision model (shared convolutional base) in Keras:

In [ ]:
from keras import layers
from keras import applications
from keras import Input

# The base image-processing model is the Xception network (convolutional base only)
xception_base = applications.Xception(weights=None,include_top=False)

In [ ]:
left_input = Input(shape=(250, 250, 3))  #The inputs are 250 × 250 RGB images.
right_input = Input(shape=(250, 250, 3)) #Calls the same vision model twice

left_features = xception_base(left_input)
right_input = xception_base(right_input)

merged_features = layers.concatenate([left_features, right_input], axis=-1)
# The merged features contain information from the right visual feed and the left visual feed. 